In [58]:
### string.fromat

import yaml
import import_ipynb
import Read_pdf as rp
import PySimpleGUI as sg


def display_template(data):
    topic = data.keys()
    print('--------------模板列表-----------------')
    for item in topic:
        print(item + ':')
        for sub_item in data[item]:
            print(' -' + sub_item)

def get_variable(text):#抓取{ 及 }的位置並取出裏頭的字串變成dic的key
    my_pos = []
    my_pos2 = []
    pos = 0
    pos2 = 0
    while True:
        pos = text.find('{', pos)
        pos2 = text.find('}', pos2)
        if pos2 == -1:
            break
        my_pos.append(pos)
        my_pos2.append(pos2)
        pos += 1
        pos2 +=1

    keys = []
    for start,end in zip(my_pos, my_pos2):
        keys.append(text[start+1:end])
    d = dict.fromkeys(keys)
    del my_pos
    del my_pos2
    del keys
    return d

def sg_listbox(text, **option):
    option.setdefault('title', '套版工具')
    li = text.split('/')
    layout = [
                [sg.Text('請選擇以下詞句:')],
                [sg.Listbox(values=li, size=(25, 5))],
                [sg.Submit(tooltip='送出選擇'), sg.Cancel()],
            ]

    # create the window`
    window = sg.Window(option['title'], layout)

    event, values = window.read()
    window.close()
    return values[0]


def bid_value(bid, dic, popup):#將{}內的字詞為key，判斷key決定value
    
    if bid.currency == '人民幣':
        prefixCur = 'RMB'
        suffixCur = ''
    else:
        prefixCur =''
        suffixCur = '元'

    def name_format(var):
        return '「' + str(var) +'」'

    def price_format(var):
        var = str(var).replace(' ', '')
        return prefixCur + str(var) + suffixCur
    
    select_unNone = lambda final, quo : final if final != None else quo
    
    for key in dic:
        if key == '次別':
            dic[key] = bid.sn
        elif key == '決包廠商':
            dic[key] = name_format(bid.award_vnd)
        elif key == '追加減':
            dic[key] = bid.change
        elif key == '公司別':
            dic[key] = ref_comp_code(bid)
        elif key == '廠區':
            dic[key] = bid.id[1:3]
        elif key == '預算金額':
            dic[key] = price_format(bid.budget)
        elif key == '報價金額':
            dic[key] = price_format(bid.vnd[bid.cheapest_sn]['quo'])
        elif key == '最後金額':
            dic[key] = price_format(select_unNone(bid.vnd[bid.cheapest_sn]['finalPrice'], \
                                                  bid.vnd[bid.cheapest_sn]['quo']))
        elif key == '議價金額':
            if '由/維持原報價' in dic and dic['由/維持原報價'] == '維持原報價':
                dic[key] = ''
            else:
                dic[key] = price_format(bid.vnd[bid.cheapest_sn]['finalPrice'])
            
        elif key == '差異':
            dic[key] = bid.dif_text(bid.cheapest_sn)
        elif key == '工程名稱':
            dic[key] = bid.name
        elif key == '最低廠商':
            dic[key] = name_format(bid.vnd[bid.cheapest_sn]['name'])
        elif key == '報價家數':
                dic[key] = len(bid.vnd)
        elif key == '及催報':
            dic[key] = ''
            for vnd in bid.vnd:
                if vnd['type'] == '後報' or vnd['type'] == '後加':
                    dic[key] = '及催報'
                    break
        
        elif key == '議價狀況':
            sum_bargain_times = 0 #議價總次數
            for vnd in bid.vnd:
                sum_bargain_times += int(vnd['bargain_times'])
                
            if bid.cheapest_sn == 0 and int(bid.vnd[bid.cheapest_sn]['bargain_times']) == sum_bargain_times:
                dic[key] = '經洽議報價最低廠商'
            else:
                dic[key] = '議後以廠商'
        elif key == '議後':
            if bid.vnd[bid.cheapest_sn]['bargain_times'] != '0':
                dic[key] = '議後'
            else:
                dic[key] = ''
        elif key == '由/維持原報價':
            if bid.vnd[bid.cheapest_sn]['quo'] == bid.vnd[bid.cheapest_sn]['finalPrice']:
                dic[key] = '維持原報價'
            else:
                dic[key] = '由'
        elif key == '降至':
            if dic['由/維持原報價'] == '維持原報價':
                dic[key] = ''
            else:
                dic[key] = '降至'
        
        elif key == '最低':
            if dic['議價狀況'] == '議後以廠商':
                dic[key] = '最低'
            else:
                dic[key] = ''
        
        elif key == '小規模納稅人':
            rate = select_unNone(bid.vnd[bid.cheapest_sn]['finalRate'], bid.vnd[bid.cheapest_sn]['quoRate'])
            if int(rate) == 3 :
                dic[key] = '小規模納稅人'
            else:
                dic[key] = ''
        
        elif key == '廠商稅率':
            rate = select_unNone(bid.vnd[bid.cheapest_sn]['finalRate'], bid.vnd[bid.cheapest_sn]['quoRate'])
            dic[key] = rate
        elif key == '折台幣':
            p = select_unNone(bid.vnd[bid.cheapest_sn]['finalPrice'], bid.vnd[bid.cheapest_sn]['quo'])
            p = int(p.replace(',',''))
            if bid.currency == '人民幣' and p > 100000:
                dic[key] = '(NTD{}元)'.format(f'{p * 5:,}')
            else:
                dic[key] = ''
        elif key =='發票符合/洽會同意':
            setting_rate = dic['預算稅率']
            rate = select_unNone(bid.vnd[bid.cheapest_sn]['finalRate'], bid.vnd[bid.cheapest_sn]['quoRate'])
            
            if setting_rate == bid.vnd[bid.cheapest_sn]['finalRate']:
                dic[key] = '符合'
            elif rate == '3' :
                dic[key] = ''
            elif setting_rate == '{預算稅率}':
                dic[key] = '{發票符合/洽會同意}'
            else:
                dic[key] = '(洽會工程部門確認同意)'
        
        elif key == '回議':
            if bid.cheapest_sn == 0:
                dic[key] = ''
            else:
                if bid.vnd[0]['finalPrice'] == None:
                    dic[key] = '，回議廠商{name}未回覆，不再回議'.format(name = name_format(bid.vnd[0]['name']))
                else:
                    dic[key] = '，回議廠商{name}回覆{price}價高'.format(name = name_format(bid.vnd[0]['name']),\
                                                              price = price_format(bid.vnd[0]['finalPrice']))
        elif key == '差異原因':
            dic[key] = '{差異原因}'
        
        else:
            if popup:
                sg.theme('LightGreen')
                if '/' in key:
                    dic[key] = sg_listbox(key)[0] #sg_listbox回傳list，但是單選所以直接設0
                else:
                    dic[key] = sg.popup_get_text('請輸入「{}」的值:'.format(key), '套版工具')
            else:
                dic[key] = '{'+ key + '}'

    return dic


def use_template(bidcase ,data , **kwargs):
    kwargs.setdefault('easymode', False)
    kwargs.setdefault('popup', False)

    is_bargain = lambda n :'有議價' if n else '無議價'
    is_overbudget = lambda n :'(超預算)' if n else '(低預算)'


    a = ''
    my_mode = is_bargain(bidcase.is_bargain)

    if bidcase.type == '一般案' and bidcase.is_bargain:
        a = is_overbudget(bidcase.is_overBudget(bidcase.cheapest_sn))
    elif kwargs['easymode'] and bidcase.type != '一般案':
        my_mode = '簡易版'

    text = data[bidcase.type][my_mode + a]
    print('套版選擇:[{bid_type}][{bid_mode}]'.format(bid_type = bidcase.type, bid_mode = my_mode + a))

    dict_tags = get_variable(text)
    dict_tags = bid_value(bidcase, dict_tags, kwargs['popup'])

    text = text.format(**dict_tags) 
    return text

def format_text(text, bidcase):
    dict_tags = get_variable(text)
    dict_tags = bid_value(bidcase, dict_tags, True)
    text = text.format(**dict_tags)
    return text

def ref_comp_code(bid, file = None):
    if file == None:
        file = 'comp_code.yaml'
    with open(file, 'r', encoding = 'utf8') as stream:
        comp_code = yaml.load(stream, Loader = yaml.FullLoader)
    
    area = 'TW'
    code = bid.comp + bid.id[1:3]
    if bid.currency == '人民幣':
        area = 'CN'
        code = bid.comp
        
    try:
        return comp_code[area][code]
    except KeyError:
        text = sg.popup_get_text('字詞庫comp_code.yaml缺少對應的代碼「{a}」，請直接輸入:\n工程名稱:{b}'
                                 .format(a = code, b = bid.name) ,'套版工具', font = 'jengHei 11')
        comp_code[area].update( {code : text} )
        with open(file, 'w', encoding = 'utf8') as f:
            yaml.dump(comp_code, f, default_flow_style=False, indent=2, allow_unicode=True)
        print('comp_code.yaml寫入{a}={b}'.format(a = code, b = text))
        return text
    
if __name__ == '__main__':
        
    file_yaml = 'template_tw.yaml'
    with open(file_yaml, 'r', encoding = 'utf8') as stream:
        yaml_data = yaml.load(stream, Loader = yaml.FullLoader)
        
#     display_template(yaml_data)        

    pdf_file = 'd:/autosub_source/tw2.pdf'
    bidcase = rp.open(pdf_file)
    
    
    print('----------------套版後-------------------')
    text = use_template(bidcase, yaml_data, easymode = False, popup = False)
    print(text)
    
    print('----------------更新未取代字詞-------------------')
    text = format_text(text, bidcase)
    print(text)
    
    
    

    
    

----------------套版後-------------------
套版選擇:[變更案][有議價]
1.本案為01次變更案，預算編列追加54,022元。
2.經洽議原承商「笠勛工程」後由58,972元降至54,022元，同預算價格。
3.擬以54,022元向「笠勛工程」辦理追加。

----------------更新未取代字詞-------------------
1.本案為01次變更案，預算編列追加54,022元。
2.經洽議原承商「笠勛工程」後由58,972元降至54,022元，同預算價格。
3.擬以54,022元向「笠勛工程」辦理追加。

